In [48]:
import pandas as pd


In [49]:
import numpy as np

In [50]:
fake_data = pd.read_csv("C:/Users/HP/Documents/GitHub/Fake-news-detection/Fake.csv")
fake_data.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [51]:
true_data = pd.read_csv("C:/Users/HP/Documents/GitHub/Fake-news-detection/True.csv")
true_data.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


Removing the unnecessary columns from the datasets

In [52]:
true_data = true_data.drop(["title", "subject", "date"], axis=1)
fake_data = fake_data.drop(["title", "subject", "date"], axis=1)

Adding a new column in both datasets such that for the true dataset the new column "true/fake" will have value 1 shwowing it is true and value 0 for the fake dataset

In [53]:
true_data["true/fake"]=1
fake_data["true/fake"]=0
true_data.head()

,text,true/fake
0,WASHINGTON (Reuters) - The head of a conservat...,1
1,WASHINGTON (Reuters) - Transgender people will...,1
2,WASHINGTON (Reuters) - The special counsel inv...,1
3,WASHINGTON (Reuters) - Trump campaign adviser ...,1
4,SEATTLE/WASHINGTON (Reuters) - President Donal...,1


Checking for NA values

In [54]:
true_data.isnull().sum()
fake_data.isnull().sum()

text         0
true/fake    0
dtype: int64

Combining the two datasets into a single dataset

In [55]:
main_data = pd.concat([true_data,fake_data],axis=0)
main_data = main_data.sample(frac=1)
main_data

,text,true/fake
18698,A caller to British radio host Katie Hopkins w...,0
9161,WASHINGTON (Reuters) - U.S. Secretary of State...,1
15891,BEIRUT (Reuters) - An Israeli air strike targe...,1
17175,MANILA (Reuters) - Philippine law enforcement ...,1
9129,(Reuters) - A federal judge refused to dismiss...,1
...,...,...
4224,TOKYO (Reuters) - U.S. Commerce Secretary Wilb...,1
4485,Kansas is bleeding money and jobs and Governor...,0
21242,KAMPALA (Reuters) - South Sudanese soldiers ar...,1
3150,Republicans treated President Obama like shit ...,0


Splitting the data intop training and testing data

In [56]:
from sklearn.model_selection import train_test_split
X = main_data["text"]
y = main_data["true/fake"]

In [57]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2,  random_state=42)

Import necessary packages foe stopwords removal,lemmatization and vectorization

In [58]:
import nltk 
nltk.download("stopwords")
from nltk.corpus import stopwords, wordnet
stopwords.words("english")
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download("wordnet")
nltk.download("punkt")
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

Applying stopword removal,lemmatization and vectorization

In [59]:
import re
def get_wordnet_pos(tag):
    if tag.startswith("J"):
        return wordnet.ADJ
    elif tag.startswith("V"):
        return wordnet.VERB
    elif tag.startswith("N"):
        return wordnet.NOUN
    elif tag.startswith("R"):
        return wordnet.ADV
    else:
        return wordnet.NOUN
    
def wordNetLemmatizer(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text) #uppercase non-word character
    text = re.sub('https?://\S+|www\.\S+', '', text) #links
    text = re.sub('<.*?>+', '', text) 
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text) 
    text = re.sub(r'[^\w\s]', '', text)
    wnl = WordNetLemmatizer()
    tokens = word_tokenize(text)
    wordsTags  = nltk.pos_tag(tokens)
    return [wnl.lemmatize(word, pos = get_wordnet_pos(tag)) for word, tag in wordsTags]

In [61]:
vect = TfidfVectorizer( tokenizer=wordNetLemmatizer,stop_words="english")
X_train = vect.fit_transform(X_train)
X_test = vect.transform(X_test)

c:\Users\HP\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['far', 'make', 'u'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


Model Building

In [62]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [63]:
from xgboost import XGBClassifier
xg = XGBClassifier()
xg.fit(X_train, y_train)
y_predxg = xg.predict(X_test)

In [67]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_predxg))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4753
           1       1.00      1.00      1.00      4227

    accuracy                           1.00      8980
   macro avg       1.00      1.00      1.00      8980
weighted avg       1.00      1.00      1.00      8980



In [68]:
from sklearn.model_selection import cross_val_score
acc = cross_val_score(xg, X_train, y_train, cv = 10)
print(acc.mean())

0.9972715366118836


In [69]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=10)
rf.fit(X_train, y_train)
y_predrf = rf.predict(X_test)

In [70]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_predrf))

              precision    recall  f1-score   support

           0       0.96      0.99      0.98      4753
           1       0.99      0.96      0.97      4227

    accuracy                           0.97      8980
   macro avg       0.98      0.97      0.97      8980
weighted avg       0.97      0.97      0.97      8980



In [73]:
from sklearn.model_selection import cross_val_score
acc = cross_val_score(rf, X_train, y_train, cv = 10)
print(acc.mean())

0.9647254116483983


In [71]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train, y_train)
y_predlr = lr.predict(X_test)

In [72]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_predlr))

              precision    recall  f1-score   support

           0       0.99      0.98      0.99      4753
           1       0.98      0.99      0.99      4227

    accuracy                           0.99      8980
   macro avg       0.99      0.99      0.99      8980
weighted avg       0.99      0.99      0.99      8980



In [74]:
from sklearn.model_selection import cross_val_score
acc = cross_val_score(lr, X_train, y_train, cv = 10)
print(acc.mean())

0.9839635822419199


XG Boost Classifier has the best accuracy. So we use it for prediction.